<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_5_tuning_pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    747      0 --:--:-- --:--:-- --:--:--   745
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-5.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [47]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-5.validation.csv",header=None)
valid.head()

,0,1
0,2,heel pain
1,2,heel pain
2,5,increase my heart rate
3,5,tachycardia
4,8,severe arthritis


In [48]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-5.test.csv",header=None)
test.head()

,0,1
0,0,fear
1,0,scared to death
2,2,heel pain
3,2,heels hurt
4,2,pain in my right heel


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_pubmed_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [53]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [57]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.451601,4.819250,0.228663,11:28


In [58]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.652851,4.508337,0.259428,11:25


In [59]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.360331,4.471601,0.262710,11:19


In [60]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.303822,4.444621,0.265572,11:26


In [0]:
learn.save_encoder('pubmed_fold5_first')

In [62]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_fold5_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.804157,3.730101,0.380587,01:14


In [63]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.951506,3.220301,0.432950,01:27


In [64]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.431451,2.976512,0.477650,03:25


In [65]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.164127,2.755603,0.518519,03:48
1,2.011907,2.650838,0.530013,03:25


In [66]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.875376,2.587925,0.547893,03:22
1,1.726189,2.464661,0.563218,03:39


In [67]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.669575,2.423440,0.572158,03:30
1,1.554450,2.343093,0.581098,03:49


In [68]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.540753,2.328458,0.591315,03:37
1,1.438512,2.273478,0.590038,03:32


In [69]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.386107,2.288323,0.600255,03:35
1,1.378470,2.225515,0.595147,03:38


In [70]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.278662,2.227120,0.596424,03:23
1,1.348319,2.247480,0.605364,03:41
2,1.206970,2.200078,0.615581,03:26
3,1.159680,2.179178,0.611750,03:36


In [71]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.189113,2.204293,0.623244,03:34
1,1.193646,2.224418,0.614304,03:32
2,1.133320,2.189553,0.623244,03:23
3,1.031496,2.189002,0.628352,03:42


In [72]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.065797,2.233157,0.619413,03:20
1,1.103550,2.202226,0.614304,03:38
2,1.043616,2.204245,0.625798,03:45
3,0.981358,2.171540,0.621967,03:30


In [73]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.068790,2.257332,0.625798,03:34
1,1.112239,2.259607,0.621967,03:37
2,1.033505,2.249902,0.623244,03:41
3,0.978205,2.212292,0.633461,03:44


In [74]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.978666,2.215109,0.630907,03:36
1,1.072037,2.292361,0.628352,03:46
2,1.014611,2.178313,0.633461,03:46
3,0.940759,2.211622,0.633461,03:34


In [75]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.975811,2.290989,0.618135,03:44
1,0.999258,2.271367,0.628352,03:35
2,0.964664,2.281084,0.634738,03:26
3,0.902659,2.264312,0.633461,03:24


In [76]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.964455,2.331241,0.638570,03:36
1,0.950026,2.340555,0.639847,03:44
2,0.926437,2.282787,0.632184,03:47
3,0.864966,2.292714,0.633461,03:38


In [0]:
# save the best model

learn.save_encoder('ask-5-tuning-pubmed')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [79]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[182, 283, 260, 265, 211, 8, 8, 8, 182, 16, 16, 16, 19, 27, 27, 27, 816, 197, 289, 289, 37, 37, 37, 37, 37, 741, 38, 450, 49, 182, 49, 49, 49, 100, 53, 53, 53, 265, 182, 182, 182, 470, 763, 265, 846, 129, 209, 864, 79, 289, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 260, 182, 182, 182, 80, 80, 147, 455, 238, 318, 92, 94, 94, 94, 94, 94, 94, 95, 455, 764, 182, 98, 100, 100, 100, 100, 100, 100, 100, 100, 265, 103, 182, 712, 289, 107, 88, 108, 108, 111, 730, 111, 111, 111, 111, 289, 172, 510, 182, 119, 455, 265, 182, 182, 125, 455, 125, 630, 125, 125, 125, 125, 135, 182, 89, 165, 344, 449, 289, 182, 157, 865, 202, 174, 166, 166, 166, 171, 172, 172, 289, 172, 172, 864, 864, 864, 864, 864, 214, 182, 318, 696, 182, 165, 904, 904, 218, 318, 646, 183, 183, 183, 183, 183, 183, 183, 182, 182, 182, 190, 190, 190, 604, 700, 190, 190, 190, 190, 190, 190, 265, 314, 182, 180, 1

In [81]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

449
0.5184757505773672
